In [1]:
from gurobipy import Model, GRB, quicksum
import math

In [ ]:
def _build_graph(edges):
    nodes = set()
    for i,j in edges:
        nodes.add(i); nodes.add(j)
    out_arcs = {i: [] for i in nodes}
    in_arcs  = {i: [] for i in nodes}
    for (i,j) in edges:
        out_arcs[i].append((i,j))
        in_arcs[j].append((i,j))
    return nodes, out_arcs, in_arcs

def _extract_path_from_binary_flow(alpha_sol, out_arcs, source, sink):
    path = []
    cur = source
    visited = set()
    while cur != sink and cur not in visited:
        visited.add(cur)
        nxt = None
        for e in out_arcs[cur]:
            if alpha_sol[e] > 0.5:
                nxt = e
                break
        if nxt is None:
            break
        path.append(nxt)
        cur = nxt[1]
    return path

In [2]:
import gurobipy as gp
from gurobipy import GRB

def solve_SHIMRITS_adversary(
    bar_p,
    hat_p,
    edges,
    source,
    sink,
    Gamma,
    gurobi_output=False,
):
    n = len(bar_p)

    nodes = set()
    for i, j in edges:
        nodes.add(i)
        nodes.add(j)
    V = sorted(nodes)
    
    #Create Pi group
    Pi = {0.0}
    for i in range(n):
        if hat_p[i] > 0:
            Pi.add(float(hat_p[i]))
    Pi = sorted(Pi)
    
    #The flow variables
    out_arcs = {i: [] for i in V}
    in_arcs = {i: [] for i in V}
    for (i, j) in edges:
        out_arcs[i].append((i, j))
        in_arcs[j].append((i, j))

    model = gp.Model("adv_compact")
    if not gurobi_output:
        model.Params.OutputFlag = 0

    phi = model.addVars(edges, lb=0, ub=1, name="phi")
    alpha = model.addVars(range(n), vtype=GRB.BINARY, name="alpha")
    t = model.addVar(lb=-GRB.INFINITY, name="t")

    model.update()


    
    for i in V:
        if i == source:
            model.addConstr(gp.quicksum(phi[e] for e in out_arcs[i]) == 1)
        elif i == sink:
            model.addConstr(gp.quicksum(phi[e] for e in in_arcs[i]) == 1)
        else:
            model.addConstr(
                gp.quicksum(phi[e] for e in out_arcs[i]) -
                gp.quicksum(phi[e] for e in in_arcs[i])
                == 0
            )

    for i in range(n):
        model.addConstr(
            alpha[i] == gp.quicksum(phi[e] for e in out_arcs.get(i, [])),
            name=f"alpha_def_{i}",
        )

    for pi in Pi:
        expr = pi * Gamma
        for i in range(n):
            expr += max(hat_p[i] - pi, 0.0) * alpha[i]
        model.addConstr(t <= expr, name=f"pi_constraint_{pi}")

    obj = t + gp.quicksum(alpha[i] * bar_p[i] for i in range(n))
    model.setObjective(obj, GRB.MAXIMIZE)

    model.optimize()

    if model.status != GRB.OPTIMAL:
        return dict(
            best_value=None,
            best_pi=None,
            path_arcs=None,
            phi=None,
            alpha=None,
        )

    phi_sol = {e: phi[e].X for e in edges}
    alpha_sol = [alpha[i].X for i in range(n)]
    best_value = model.ObjVal

    path_arcs = []
    current = source
    visited = set()
    while current != sink and current not in visited:
        visited.add(current)
        nxt_arc = None
        for e in out_arcs[current]:
            if phi_sol.get(e, 0) > 0.5:
                nxt_arc = e
                break
        if nxt_arc is None:
            break
        path_arcs.append(nxt_arc)
        current = nxt_arc[1]

    # compute minimizer pi after solving
    t_star = t.X
    best_pi = None
    best_f_pi = float("inf")
    for pi in Pi:
        f_pi = pi * Gamma
        for i in range(n):
            f_pi += max(hat_p[i] - pi, 0.0) * alpha_sol[i]
        if f_pi < best_f_pi + 1e-6:
            best_f_pi = f_pi
            best_pi = pi

    return dict(
        best_value=best_value,
        best_pi=best_pi,
        path_arcs=path_arcs,
        phi=phi_sol,
        alpha=alpha_sol,
    )


In [3]:
def solve_adversary_bruni(
    bar_p,
    hat_p,
    edges,
    source,
    sink,
    Gamma,
    gurobi_output=False,
):
    """
    Bruni-style adversarial subproblem formulation (Section 4.2):

        max sum_(i,j) bar_p[i] * alpha_ij + hat_p[i] * w_ij

    s.t. alpha defines a path from source to sink,
         w_ij <= xi_i,
         w_ij <= alpha_ij,
         sum_i xi_i <= Gamma,
         alpha_ij in {0,1},
         0 <= xi_i <= 1, w_ij >= 0.

    Parameters
    ----------
    bar_p : list[float]
        Nominal durations bar_p[i] for each activity node i.
    hat_p : list[float]
        Deviations hat_p[i] for each activity node i.
    edges : list[tuple[int, int]]
        List of directed arcs (i, j) in the extended project network.
    source : int
        Index of the source node.
    sink : int
        Index of the sink node.
    Gamma : float or int
        Budget parameter.
    gurobi_output : bool, optional
        If True, Gurobi's output flag is enabled.

    Returns
    -------
    result : dict
        {
          "value": float,
          "path_arcs": list[(i, j)],
          "alpha": dict[(i,j)] -> float,
          "xi": dict[i] -> float,
          "w": dict[(i,j)] -> float
        }
    """
    n = len(bar_p)
    nodes, out_arcs, in_arcs = _build_graph(edges)

    model = gp.Model("adversary_bruni")
    model.Params.OutputFlag = 1 if gurobi_output else 0

    # Binary path variables alpha_ij
    alpha = {   
        (i, j): model.addVar(vtype=GRB.BINARY, name=f"alpha_{i}_{j}")
        for (i, j) in edges
    }

    # Continuous w_ij >= 0
    w = {
        (i, j): model.addVar(lb=0.0, vtype=GRB.CONTINUOUS, name=f"w_{i}_{j}")
        for (i, j) in edges
    }

    # Activity delay variables xi_i in [0, 1]
    xi = {
        i: model.addVar(lb=0.0, ub=1.0, vtype=GRB.CONTINUOUS, name=f"xi_{i}")
        for i in nodes
    }

    model.update()

    # Flow constraints: single path from source to sink
    # 1) Source: sum of outgoing arcs is 1
    model.addConstr(
        gp.quicksum(alpha[(i, j)] for (i, j) in out_arcs[source]) == 1,
        name="source_flow",
    )

    # 2) Sink: sum of incoming arcs is 1
    model.addConstr(
        gp.quicksum(alpha[(i, j)] for (i, j) in in_arcs[sink]) == 1,
        name="sink_flow",
    )

    # 3) Intermediate nodes: flow conservation
    for i in nodes:
        if i in (source, sink):
            continue
        model.addConstr(
            gp.quicksum(alpha[(i, j)] for (i, j) in out_arcs[i])
            - gp.quicksum(alpha[(u, v)] for (u, v) in in_arcs[i])
            == 0,
            name=f"flow_balance_{i}",
        )

    # Linking constraints w_ij <= xi_i and w_ij <= alpha_ij
    for (i, j) in edges:
        model.addConstr(w[(i, j)] <= xi[i], name=f"w_le_xi_{i}_{j}")
        model.addConstr(w[(i, j)] <= alpha[(i, j)], name=f"w_le_alpha_{i}_{j}")

    # Budget constraint: sum_i xi_i <= Gamma
    model.addConstr(
        gp.quicksum(xi[i] for i in nodes) <= Gamma,
        name="budget_Gamma",
    )

    # Objective: sum_(i,j) bar_p[i] * alpha_ij + hat_p[i] * w_ij
    obj_expr = gp.LinExpr()
    for (i, j) in edges:
        if 0 <= i < n:
            obj_expr += bar_p[i] * alpha[(i, j)] + hat_p[i] * w[(i, j)]

    model.setObjective(obj_expr, GRB.MAXIMIZE)
    model.optimize()

    if model.status != GRB.OPTIMAL:
        raise RuntimeError(
            f"Gurobi did not find an optimal solution. Status: {model.status}"
        )

    value = model.ObjVal
    alpha_sol = {e: alpha[e].X for e in edges}
    xi_sol = {i: xi[i].X for i in nodes}
    w_sol = {e: w[e].X for e in edges}

    path_arcs = _extract_path_from_binary_flow(alpha_sol, out_arcs, source, sink)

    return {
        "value": value,
        "path_arcs": path_arcs,
        "alpha": alpha_sol,
        "xi": xi_sol,
        "w": w_sol,
    }

**MASTER**

In [4]:
import gurobipy as gp
from gurobipy import GRB
def _get_b_node(b_node, i, k):
    """
    Helper to read b_{ik}. Supports:
      - dict[(i,k)] -> value
      - list-of-lists b_node[i][k]
    """
    if isinstance(b_node, dict):
        return float(b_node.get((i, k), 0.0))
    return float(b_node[i][k])

def _get_b_supply(b_supply, k):
    """
    Helper to read b_k. Supports:
      - dict[k] -> value
      - list/tuple b_supply[k]
    """
    if isinstance(b_supply, dict):
        return float(b_supply[k])
    return float(b_supply[k])

def create_master_problem(
    V,
    E,
    P_list,
    R,
    b_node,
    b_supply,
    M_prec=1e6,
    M_flow=1e6,
    gurobi_output=False,
):
    """
    Build the initial master problem for a given scenario set P_list.

    V       : list of nodes (0 is source, V[-1] is sink n+1)
    E       : list of fixed precedence arcs (i,j) -> x_ij = 1
    P_list  : list of duration vectors p^h; each p^h has len(p^h) == len(V)
    R       : list of resource indices (e.g. [0,1,...])
    b_node  : node-by-resource balances b_{ik}, as dict[(i,k)] or 2D list [i][k]
    b_supply: per-resource supply b_k, as dict[k] or list
    M_prec  : big-M for precedence constraints
    M_flow  : big-M for capacity constraints 0 <= f_ijk <= M_flow * x_ij

    Returns:
        m    : Gurobi model
        x    : x[i,j] binary policy arcs
        S    : dict of S[(h,i)] variables
        rho  : robust makespan variable
        f    : f[i,j,k] flow variables
    """
    m = gp.Model("master_rcpsp")
    if not gurobi_output:
        m.Params.OutputFlag = 0

    source = V[0]
    sink = V[-1]
    H = len(P_list)        # number of scenarios
    nV = len(V)

    # --- Decision variables ---

    # x_ij ∈ {0,1}
    x = m.addVars(V, V, vtype=GRB.BINARY, name="x")

    # S^h_i ≥ 0  (we store them in a dict S[(h,i)])
    S = {}
    for h, p in enumerate(P_list):
        for i in V:
            S[(h, i)] = m.addVar(lb=0.0, name=f"S[{h},{i}]")

    # ρ ≥ 0
    rho = m.addVar(lb=0.0, name="rho")

    # f_ijk ≥ 0
    f = m.addVars(V, V, R, lb=0.0, name="f")

    # --- Objective ---
    # min ρ
    m.setObjective(rho, GRB.MINIMIZE)

    # --- Constraints ---

    # (eq. rho_ge_S)  ρ ≥ S^h_{sink}  for all scenarios h
    for h in range(H):
        m.addConstr(rho >= S[(h, sink)], name=f"rho_lb[{h}]")

    # (eq. prec_scen)  S^h_j ≥ S^h_i + p^h_i - M(1 - x_ij)
    for h, p in enumerate(P_list):
        for i in V:
            p_hi = float(p[i])
            for j in V:
                if i == j:
                    continue
                m.addConstr(
                    S[(h, j)] >= S[(h, i)] + p_hi - M_prec * (1 - x[i, j]),
                    name=f"prec[{i},{j},{h}]"
                )

    # (eq. nonneg_S) is enforced by lb=0.0 on S^h_i

    # (eq. f_nonneg) is enforced by lb=0.0 on f_ijk

    # (eq. flow_conserv)  internal nodes: sum_j f_{j i k} = sum_j f_{i j k} = b_{ik}
    for k in R:
        for i in V:
            if i in (source, sink):
                continue
            inflow = gp.quicksum(f[j, i, k] for j in V if j != i)
            outflow = gp.quicksum(f[i, j, k] for j in V if j != i)
            b_ik = _get_b_node(b_node, i, k)
            m.addConstr(inflow == b_ik, name=f"flow_in[{i},{k}]")
            m.addConstr(outflow == b_ik, name=f"flow_out[{i},{k}]")

    # (eq. flow_supply)  source/sink: sum_j f_{0 j k} = sum_j f_{j,n+1,k} = b_k
    for k in R:
        b_k = _get_b_supply(b_supply, k)
        m.addConstr(
            gp.quicksum(f[source, j, k] for j in V if j != source) == b_k,
            name=f"supply_source[{k}]"
        )
        m.addConstr(
            gp.quicksum(f[j, sink, k] for j in V if j != sink) == b_k,
            name=f"supply_sink[{k}]"
        )

    # (eq. cap_flow)  0 ≤ f_{ijk} ≤ M * x_ij
    for k in R:
        for i in V:
            for j in V:
                if i == j:
                    # if you want, you can force no self-flow:
                    m.addConstr(f[i, j, k] == 0.0, name=f"no_self_flow[{i},{k}]")
                else:
                    m.addConstr(
                        f[i, j, k] <= M_flow * x[i, j],
                        name=f"cap[{i},{j},{k}]"
                    )

    # (eq. x_fixed)  x_{ij} = 1 for (i,j) in E
    for (i, j) in E:
        m.addConstr(x[i, j] == 1, name=f"x_fixed[{i},{j}]")

    # (eq. x_bin) is enforced by vtype=GRB.BINARY

    m.update()
    return m, x, S, rho, f



def add_scenario_to_master(
    m,
    x,
    S,
    rho,
    p_vec,
    h_new,
    V,
    M_prec=1e6,
):
    """
    Add a new scenario h_new with durations p_vec to an existing master model.

    m     : existing Gurobi model
    x     : x[i,j] binary vars (shared across scenarios)
    S     : dict of S[(h,i)] vars; this function will extend it
    rho   : robust makespan var
    p_vec : list/array of durations p^{h_new}_i (len == len(V))
    h_new : integer index of the new scenario (e.g. current number of scenarios)
    V     : list of nodes
    M_prec: big-M for precedence constraints
    """
    sink = V[-1]

    # Create S^{h_new}_i ≥ 0 for all i
    for i in V:
        S[(h_new, i)] = m.addVar(lb=0.0, name=f"S[{h_new},{i}]")

    # Add constraint ρ ≥ S^{h_new}_{sink}
    m.addConstr(rho >= S[(h_new, sink)], name=f"rho_lb[{h_new}]")

    # Add precedence constraints for scenario h_new:
    # S^{h_new}_j ≥ S^{h_new}_i + p^{h_new}_i - M*(1 - x_ij)
    for i in V:
        p_hi = float(p_vec[i])
        for j in V:
            if i == j:
                continue
            m.addConstr(
                S[(h_new, j)] >= S[(h_new, i)] + p_hi - M_prec * (1 - x[i, j]),
                name=f"prec[{i},{j},{h_new}]"
            )

    m.update()



def extract_policy_edges_from_x(x, V, tol=0.5):
    """
    Extract the current earliest-start policy X^q from the Gurobi variables x[i,j].

    Parameters
    ----------
    x   : Gurobi tupledict of binary vars x[i,j]
    V   : list of nodes (e.g. [0,1,...,n])
    tol : threshold above which x[i,j] is considered '1'

    Returns
    -------
    policy_edges : list of (i,j) such that x[i,j] ≈ 1
    """
    policy_edges = []
    for i in V:
        for j in V:
            if i == j:
                continue
            val = x[i, j].X
            if val is not None and val > tol:
                policy_edges.append((i, j))
    return policy_edges


**Adversary**

In [5]:
import math

def _active_nodes_from_path_arcs(path_arcs, source, sink):
    nodes_on_path = set()
    for (i, j) in path_arcs:
        nodes_on_path.add(i)
        nodes_on_path.add(j)
    nodes_on_path.discard(source)
    nodes_on_path.discard(sink)
    return sorted(nodes_on_path)


def _build_p_star_from_active_nodes(active_nodes, bar_p, hat_p, Gamma, source=None, sink=None):
    """
    Worst-case realization on the chosen path:
    - sort active nodes by hat_p desc
    - allocate z_i = 1 for top floor(Gamma), z_next = frac(Gamma), rest 0
    - p*_i = bar_p[i] + z_i * hat_p[i] for i on path, else bar_p[i]
    """
    n = len(bar_p)
    p_star = [float(bar_p[i]) for i in range(n)]

    # filter nodes that are valid indices and not source/sink (just in case)
    filtered = []
    for i in active_nodes:
        if 0 <= i < n and (source is None or i != source) and (sink is None or i != sink):
            filtered.append(i)

    filtered.sort(key=lambda i: float(hat_p[i]), reverse=True)

    g_floor = int(math.floor(float(Gamma)))
    g_frac = float(Gamma) - g_floor

    for rank, i in enumerate(filtered, start=1):
        if rank <= g_floor:
            z = 1.0
        elif rank == g_floor + 1 and g_frac > 1e-12:
            z = g_frac
        else:
            z = 0.0
        p_star[i] = float(bar_p[i]) + z * float(hat_p[i])

    return p_star


def solve_SHIMRITS_adversary_meta(
    bar_p,
    hat_p,
    edges,
    source,
    sink,
    Gamma,
    gurobi_output=False,
):
    """
    Adapter to return:
      best_pi, best_duration, best_phi, active_nodes, p_star, pi_values
    """
    res = solve_SHIMRITS_adversary(
        bar_p=bar_p,
        hat_p=hat_p,
        edges=edges,
        source=source,
        sink=sink,
        Gamma=Gamma,
        gurobi_output=gurobi_output,
    )

    if res.get("best_value", None) is None:
        raise RuntimeError("SHIMRITS adversary did not return an optimal solution.")

    best_duration = float(res["best_value"])
    best_pi = res.get("best_pi", None)
    path_arcs = res.get("path_arcs", []) or []
    best_phi = res.get("phi", None)

    active_nodes = _active_nodes_from_path_arcs(path_arcs, source, sink)

    # Optional: expose Pi list (same construction as inside SHIMRITS)
    pi_values = sorted({0.0} | {float(h) for h in hat_p if float(h) > 0.0})

    p_star = _build_p_star_from_active_nodes(
        active_nodes=active_nodes,
        bar_p=bar_p,
        hat_p=hat_p,
        Gamma=Gamma,
        source=source,
        sink=sink,
    )

    return best_pi, best_duration, best_phi, active_nodes, p_star, pi_values


In [6]:
import gurobipy as gp
from gurobipy import GRB
import math


def _active_nodes_from_path_arcs(path_arcs, source, sink):
    """Return internal nodes on the selected path (exclude source/sink)."""
    nodes_on_path = set()
    for (i, j) in path_arcs:
        nodes_on_path.add(i)
        nodes_on_path.add(j)
    nodes_on_path.discard(source)
    nodes_on_path.discard(sink)
    return sorted(nodes_on_path)


def solve_adversary_bruni_meta(
    bar_p,
    hat_p,
    edges,
    source,
    sink,
    Gamma,
    gurobi_output=False,
):
    """
    Adapter around solve_adversary_bruni that returns the tuple signature expected by worst_case_rcpsp_meta:

        best_pi, best_duration, best_phi, active_nodes, p_star, pi_values

    Notes:
    - Bruni formulation has no pi, so best_pi=None and pi_values=None.
    - p_star is built using xi only on the chosen path (important!).
    """

    # --- run your existing Bruni solver ---
    res = solve_adversary_bruni(
        bar_p=bar_p,
        hat_p=hat_p,
        edges=edges,
        source=source,
        sink=sink,
        Gamma=Gamma,
        gurobi_output=gurobi_output,
    )

    best_duration = float(res["value"])         # worst-case value for X^q
    path_arcs = res.get("path_arcs", [])
    alpha_sol = res.get("alpha", {})            # edge -> {0,1}
    xi_sol = res.get("xi", {})                  # node -> [0,1]

    # --- compute active nodes on the selected path ---
    active_nodes = _active_nodes_from_path_arcs(path_arcs, source, sink)

    # --- build p_star consistent with the chosen path ---
    # only nodes on the path are allowed to "consume" Gamma in the realized scenario for this path
    n = len(bar_p)
    on_path = set(active_nodes)

    p_star = [0.0] * n
    for i in range(n):
        base = float(bar_p[i])
        dev = float(hat_p[i]) if i < len(hat_p) else 0.0
        xi_i = float(xi_sol.get(i, 0.0))
        if i in on_path:
            p_star[i] = base + dev * xi_i
        else:
            p_star[i] = base

    best_pi = None
    best_phi = alpha_sol   # use alpha as phi (binary edge selection)
    pi_values = None
    
    return best_pi, best_duration, best_phi, active_nodes, p_star, pi_values


In [7]:
def build_pi_set(hat_p):
    """
    hat_p: list of deviations hat_p[i]
    returns sorted list Pi = {0} ∪ {hat_p_i}
    """
    values = {0.0}
    for v in hat_p:
        values.add(float(v))
    return sorted(values)

def compute_durations_for_pi(pi, bar_p, hat_p):
    """
    bar_p, hat_p: lists of same length
    returns list p_pi[i] = bar_p[i] + max(hat_p[i] - pi, 0)
    """
    n = len(bar_p)
    p_pi = [0.0] * n
    for i in range(n):
        p_pi[i] = float(bar_p[i]) + max(float(hat_p[i]) - pi, 0.0)
    return p_pi


def solve_nominal_with_gurobi_simple(pi, bar_p, hat_p, edges, source, sink, gurobi_output=False):
    """
    bar_p, hat_p: lists (length n)
    edges: list of (i, j) with i,j in {0,...,n-1}
    source, sink: integers

    Model:
        min   sum_{(i,j)} -p_i(pi) * phi_ij
        s.t.  flow constraints

    Returns:
        total_duration (float): max project duration  = -adv_obj
        adv_obj (float): min value of sum -p_i(pi) phi_ij (≤ 0)
        phi_values: dict {(i,j): value}
    """
    p_pi = compute_durations_for_pi(pi, bar_p, hat_p)

    n = len(bar_p)
    nodes = list(range(n))

    model = Model("nominal_adversary")
    if not gurobi_output:
        model.Params.OutputFlag = 0

    # Decision variables phi_ij
    phi = {}
    for (i, j) in edges:
        phi[(i, j)] = model.addVar(vtype=GRB.BINARY, name=f"phi_{i}_{j}")

    model.update()

    # Objective: min sum -p_i(pi) * phi_ij
    obj_expr = quicksum(-p_pi[i] * phi[(i, j)] for (i, j) in edges)
    model.setObjective(obj_expr, GRB.MINIMIZE)

    # Flow constraints
    for i in nodes:
        outgoing = quicksum(phi[(u, v)] for (u, v) in edges if u == i)
        incoming = quicksum(phi[(u, v)] for (u, v) in edges if v == i)

        if i == source:
            model.addConstr(outgoing == 1.0, name=f"source_flow_{i}")
        elif i == sink:
            model.addConstr(incoming == 1.0, name=f"sink_flow_{i}")
        else:
            model.addConstr(outgoing - incoming == 0.0, name=f"flow_{i}")

    model.optimize()

    if model.Status != GRB.OPTIMAL:
        raise RuntimeError(f"Nominal problem not solved to optimality, status={model.Status}")

    adv_obj = model.ObjVal              # = min sum -p_i phi_ij ≤ 0
    total_duration = -adv_obj           # = max sum +p_i phi_ij ≥ 0

    phi_values = {(i, j): float(var.X) for (i, j), var in phi.items()}
    return total_duration, adv_obj, phi_values


def get_active_nodes_from_phi_simple(phi_values, n, source, sink, tol=1e-6):
    """
    n: number of nodes (nodes are 0,...,n-1)
    phi_values: dict {(i,j): value}
    returns list of nodes i (excluding source,sink) with sum_j phi_ij ≈ 1
    """
    outgoing_sum = [0.0] * n
    for (i, j), val in phi_values.items():
        outgoing_sum[i] += float(val)

    active_nodes = [
        i for i in range(n)
        if i not in (source, sink) and abs(outgoing_sum[i] - 1.0) <= tol
    ]
    return active_nodes



def build_worst_case_durations_simple(active_nodes, bar_p, hat_p, Gamma):
    """
    active_nodes: list of node indices on the robust path (excluding source,sink)
    bar_p, hat_p: lists
    Gamma: float

    p_i* = bar_p[i] + z_i * hat_p[i], with z_i ∈ {0, fractional, 1}.

    Returns:
        p_star: list of positive durations (worst-case realization).
    """
    n = len(bar_p)
    p_star = [0 for i in range(n)]

    # sort active nodes by hat_p[i] descending
    sorted_nodes = sorted(active_nodes, key=lambda i: float(hat_p[i]), reverse=True)

    floor_gamma = int(math.floor(Gamma))
    frac_gamma = float(Gamma - floor_gamma)

    for idx, i in enumerate(sorted_nodes, start=1):
        base = float(bar_p[i])
        hat = float(hat_p[i])

        if idx <= floor_gamma:
            z_i = 1.0
        elif idx == floor_gamma + 1 and frac_gamma > 0.0 and floor_gamma < len(sorted_nodes):
            z_i = frac_gamma
        else:
            z_i = 0.0

        p_star[i] = base + hat * z_i
        
    
    
    return p_star, sum(p_star)




def solve_adversary_over_all_pi_simple(bar_p, hat_p, edges, source, sink, Gamma, gurobi_output=False):
    """
    High-level adversary:

      For each pi in Pi:
        - Solve min sum -p_i(pi) phi_ij
        - Recover max project duration = -adv_obj

      Then:
        - Choose pi with minimal adv_obj (equivalently maximal project duration)
        - Extract active nodes and build p_star.

    Returns:
      best_pi
      best_duration        (positive project duration)
      best_adv_obj         (adversary objective, ≤ 0)
      best_phi
      active_nodes
      p_star               (positive worst-case durations)
      pi_values
    """
    n = len(bar_p)
    pi_values = build_pi_set(hat_p)

    best_pi = None
    best_adv_obj = math.inf      # we minimize the adversarial objective
    best_duration = None
    best_phi = None
    
    for pi in pi_values:
        total_duration, adv_obj, phi_values = solve_nominal_with_gurobi_simple(
            pi=pi,
            bar_p=bar_p,
            hat_p=hat_p,
            edges=edges,
            source=source,
            sink=sink,
            gurobi_output=gurobi_output,
        )

        # We want the "most damaging" scenario: minimal adv_obj (i.e., longest duration)
        if adv_obj < best_adv_obj:
            best_adv_obj = adv_obj
            best_duration = total_duration
            best_pi = pi
            best_phi = phi_values

    if best_phi is None:
        raise RuntimeError("No feasible pi found; check the input graph and durations.")

    active_nodes = get_active_nodes_from_phi_simple(
        phi_values=best_phi,
        n=n,
        source=source,
        sink=sink,
    )

    p_star, best_duration = build_worst_case_durations_simple(
        active_nodes=active_nodes,
        bar_p=bar_p,
        hat_p=hat_p,
        Gamma=Gamma,
    )

    return best_pi, best_duration, best_phi, active_nodes, p_star, pi_values




In [8]:
'''def worst_case_rcpsp_meta(
    V,
    E,
    R,
    bar_p,
    hat_p,
    Gamma,
    b_node,
    b_supply,
    epsilon=1e-3,
    M_prec=1e6,
    M_flow=1e6,
    max_iters=20,
    gurobi_output=False,
):
    """
    Meta-algorithm for the Worst-Case RCPSP under budgeted duration uncertainty.

    Implements Algorithm 'Worst-Case RCPSP under Budgeted Duration Uncertainty':

      Initialization:
        - choose initial duration vector p^1 in P (here: p^1 = bar_p)
        - P_hat_1 = {p^1}
        - q = 1, LB = 0, UB = +inf

      While UB - LB > epsilon:
        Step 1: Restricted Master over P_hat_q -> X^q, LB
        Step 2: Adversary -> rho_max(X^q), worst-case durations p^{q+1}
        Step 3: Update UB, and if LB != UB add new scenario and continue

    Inputs
    ------
    V        : list of nodes (0 is source, V[-1] is sink)
    E        : list of fixed precedence arcs (i,j)
    R        : list of resource indices
    bar_p    : list of nominal durations (length |V|)
    hat_p    : list of deviations (length |V|)
    Gamma    : uncertainty budget
    b_node   : node-by-resource balances b_{ik} (dict[(i,k)] or 2D list [i][k])
    b_supply : per-resource supply b_k (dict[k] or list)
    epsilon  : tolerance for stopping condition on UB - LB
    M_prec   : big-M for precedence constraints
    M_flow   : big-M for flow capacity constraints
    max_iters: safety cap on number of iterations
    gurobi_output: whether to print Gurobi logs

    Returns
    -------
    best_policy : list of arcs (i,j) representing X^*
    rho_star    : worst-case makespan (approx. = UB at termination)
    LB_hist     : list of LB values per iteration
    UB_hist     : list of UB values per iteration
    scenarios   : list of all scenario vectors p^h that were added
    """
    source = V[0]
    sink   = V[-1]

    # Initial scenario p^1: we take the nominal vector bar_p
    p1 = list(bar_p)
    P_list = [p1]

    # Build initial master over P_hat_1 = {p^1}
    m, x, S, rho, f = create_master_problem(
        V=V,
        E=E,
        P_list=P_list,
        R=R,
        b_node=b_node,
        b_supply=b_supply,
        M_prec=M_prec,
        M_flow=M_flow,
        gurobi_output=gurobi_output,
    )

    q = 1
    LB = 0.0
    UB = math.inf

    LB_hist = []
    UB_hist = []

    best_policy = None
    best_UB = UB

    for it in range(1, max_iters + 1):
        # ---- Step 1: solve restricted master over current P_hat_q ----
        m.optimize()
        if m.Status != 2:  # GRB.OPTIMAL
            raise RuntimeError(f"Master problem not optimal, status={m.Status}")

        LB = float(rho.X)
        LB_hist.append(LB)

        # Extract current earliest-start policy X^q from x
        policy_edges = extract_policy_edges_from_x(x, V, tol=0.5)
        print(f"Iter {it}: selection arcs (policy X^{q}) = {policy_edges}")  # <-- ADDED

        # Adversary graph uses base arcs E and chosen policy arcs X^q
        adversary_edges = list(set(E) | set(policy_edges))

        # ---- Step 2: adversarial worst-case for X^q ----
        best_pi, best_duration, best_phi, active_nodes, p_star, pi_values = \
            solve_adversary_over_all_pi_simple(
                bar_p=bar_p,
                hat_p=hat_p,
                edges=adversary_edges,
                source=source,
                sink=sink,
                Gamma=Gamma,
                gurobi_output=gurobi_output,
            )

        rho_max_Xq = float(best_duration)
        UB = min(UB, rho_max_Xq)
        UB_hist.append(UB)

        # Track best policy and UB so far
        if rho_max_Xq <= best_UB:
            best_UB = rho_max_Xq
            best_policy = list(policy_edges)

        # Stopping criterion
        if UB - LB <= epsilon:
            break

        # ---- Step 3: update scenario set if LB != UB ----
        # Add new scenario p^{q+1} = p_star to the master as h_new
        q += 1
        h_new = q - 1
        P_list.append(list(p_star))

        add_scenario_to_master(
            m=m,
            x=x,
            S=S,
            rho=rho,
            p_vec=p_star,
            h_new=h_new,
            V=V,
            M_prec=M_prec,
        )

    # At termination, best_policy is our X^*, best_UB approximates rho^*
    rho_star = best_UB

    return best_policy, rho_star, LB_hist, UB_hist, P_list
'''

import math

def worst_case_rcpsp_meta(
    V,
    E,
    R,
    bar_p,
    hat_p,
    Gamma,
    b_node,
    b_supply,
    adversary_type,        # "shimrits" or "bruni"
    epsilon=1e-3,
    M_prec=1e6,
    M_flow=1e6,
    max_iters=20,
    gurobi_output=False,
):
    """
    Meta-algorithm for the Worst-Case RCPSP under budgeted duration uncertainty.

    adversary_type:
        "shimrits" -> solve_SHIMRITS_adversary_meta
        "bruni"    -> solve_adversary_bruni_meta
    """

    source = V[0]
    sink   = V[-1]

    # --- choose adversary function ---
    adv_key = adversary_type.lower()
    if adv_key == "shimrits":
        adversary_fn = solve_SHIMRITS_adversary_meta
    elif adv_key == "bruni":
        adversary_fn = solve_adversary_bruni_meta
    else:
        raise ValueError("adversary_type must be 'shimrits' or 'bruni'")

    # --- initialization ---
    p1 = list(bar_p)
    P_list = [p1]

    m, x, S, rho, f = create_master_problem(
        V=V,
        E=E,
        P_list=P_list,
        R=R,
        b_node=b_node,
        b_supply=b_supply,
        M_prec=M_prec,
        M_flow=M_flow,
        gurobi_output=gurobi_output,
    )

    LB = 0.0
    UB = math.inf

    LB_hist = []
    UB_hist = []

    best_policy = None
    best_UB = math.inf

    # --- main loop ---
    for it in range(1, max_iters + 1):

        # Step 1: restricted master
        m.optimize()
        if m.Status != 2:  # GRB.OPTIMAL
            raise RuntimeError(f"Master not optimal, status={m.Status}")

        LB = float(rho.X)
        LB_hist.append(LB)

        policy_edges = extract_policy_edges_from_x(x, V, tol=0.5)

        adversary_edges = list(set(E) | set(policy_edges))

        # Step 2: adversary
        best_pi, best_duration, best_phi, active_nodes, p_star, pi_values = adversary_fn(
            bar_p=bar_p,
            hat_p=hat_p,
            edges=adversary_edges,
            source=source,
            sink=sink,
            Gamma=Gamma,
            gurobi_output=gurobi_output,
        )

        rho_max_Xq = float(best_duration)
        UB = min(UB, rho_max_Xq)
        UB_hist.append(UB)

        if rho_max_Xq <= best_UB:
            best_UB = rho_max_Xq
            best_policy = list(policy_edges)

        # stopping criterion
        if UB - LB <= epsilon:
            break

        # Step 3: add new scenario
        P_list.append(list(p_star))
        h_new = len(P_list) - 1

        add_scenario_to_master(
            m=m,
            x=x,
            S=S,
            rho=rho,
            p_vec=p_star,
            h_new=h_new,
            V=V,
            M_prec=M_prec,
        )

    rho_star = best_UB
    return best_policy, rho_star, LB_hist, UB_hist, P_list


In [9]:
import os
import time
import math
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path


# ============================================================
# 1) read psplib single-mode .sm
# ============================================================

def _ints(tokens):
    return [int(x) for x in tokens]

def _index_of_line(lines, substr):
    for i, ln in enumerate(lines):
        if substr in ln:
            return i
    raise ValueError(substr)

def _rhs(line):
    return line.split(":", 1)[1].strip()

def parse_psplib_sm(path):
    path = Path(path)
    with path.open() as f:
        lines = [ln.rstrip() for ln in f]

    nj = int(_rhs(lines[_index_of_line(lines, "jobs (incl. supersource")]))
    nr = int(_rhs(lines[_index_of_line(lines, "- renewable")]).split()[0])

    prec_off = _index_of_line(lines, "PRECEDENCE RELATIONS:") + 2
    attr_off = _index_of_line(lines, "REQUESTS/DURATIONS") + 3
    cap_off  = _index_of_line(lines, "RESOURCEAVAILABILITIES") + 2

    jobs = [f"j{i+1}" for i in range(nj)]
    resources = [f"r{k+1}" for k in range(nr)]

    succs = {}
    for ix, j in enumerate(jobs):
        toks = lines[prec_off + ix].split()
        succs[j] = [f"j{int(s)}" for s in toks[3:]]

    durations = {}
    demands = {}
    for ix, j in enumerate(jobs):
        toks = lines[attr_off + ix].split()
        durations[j] = int(toks[2])
        rd = _ints(toks[3:])
        for kk, res in enumerate(resources):
            demands[(j, res)] = rd[kk]

    cap_tokens = lines[cap_off].split()
    capacities = {resources[k]: int(cap_tokens[k]) for k in range(nr)}

    return dict(
        jobs=jobs,
        resources=resources,
        succs=succs,
        durations=durations,
        demands=demands,
        capacities=capacities,
    )


# ============================================================
# 2) convert to robust RCPSP sets
# ============================================================

def psplib_to_rcpsp_sets(data):
    jobs = data["jobs"]
    resources = data["resources"]
    succs = data["succs"]
    durations = data["durations"]
    demands = data["demands"]
    caps = data["capacities"]

    job_index = {j: i for i, j in enumerate(jobs)}
    V = list(job_index.values())
    V = sorted(V)

    E = []
    for jf, scl in succs.items():
        i = job_index[jf]
        for jt in scl:
            E.append((i, job_index[jt]))

    R = resources[:]  # resource names like r1,r2,...

    bar_p = {job_index[j]: float(durations[j]) for j in jobs}
    hat_p = {i: bar_p[i] / 2.0 for i in V}

    r = {}
    for j in jobs:
        i = job_index[j]
        for res in resources:
            r[(i, res)] = float(demands[(j, res)])

    b_supply = {res: float(caps[res]) for res in resources}

    return V, E, R, bar_p, hat_p, r, b_supply


# ============================================================
# 3) build b_node from demands (for your master)
# ============================================================

def build_b_node(V, R, r_demands):
    # b_node[(i,k)] = demand of resource k at node i
    return {(i, k): float(r_demands.get((i, k), 0.0)) for i in V for k in R}


# ============================================================
# 4) txt writer
# ============================================================

def write_meta_results_txt(df, out_path):
    df = df.copy()
    sort_cols = [c for c in ["dataset", "file", "Gamma", "adversary"] if c in df.columns]
    if sort_cols:
        df = df.sort_values(sort_cols).reset_index(drop=True)

    with open(out_path, "w", encoding="utf-8") as f:
        f.write("Worst-case RCPSP meta-algorithm results\n")
        f.write("=" * 90 + "\n\n")

        for _, row in df.iterrows():
            f.write(
                f"dataset={row.get('dataset')} | file={row.get('file')} | "
                f"n={row.get('n')} | Gamma={row.get('Gamma')} | "
                f"adversary={row.get('adversary')} | "
                f"runtime={row.get('runtime'):.6f}s | "
                f"iters={row.get('iters')} | "
                f"rho_star={row.get('rho_star')}\n"
            )
            pol = row.get("best_policy")
            f.write(f"best_policy={pol}\n")
            f.write("-" * 90 + "\n")

    print(f"Wrote: {out_path}")


# ============================================================
# 5) benchmark meta over folder (Gammas fixed: 3,5,7)
# ============================================================

def benchmark_meta(folder, gammas=(3, 5, 7), epsilon=1e-3, max_iters=20, gurobi_output=False):
    rows = []
    files = [f for f in os.listdir(folder) if f.endswith(".sm")]

    for fn in files:
        full = os.path.join(folder, fn)
        data = parse_psplib_sm(full)
        V, E, R, bar_p_dict, hat_p_dict, r_demands, b_supply = psplib_to_rcpsp_sets(data)

        n = len(V)

        # lists for meta algorithm (assumes nodes are 0..n-1)
        bar_p = [bar_p_dict[i] for i in range(n)]
        hat_p = [hat_p_dict[i] for i in range(n)]

        b_node = build_b_node(V, R, r_demands)

        for Gamma in gammas:
            for adv in ("shimrits", "bruni"):
                t0 = time.perf_counter()
                best_policy, rho_star, LB_hist, UB_hist, P_list = worst_case_rcpsp_meta(
                    V=V,
                    E=E,
                    R=R,
                    bar_p=bar_p,
                    hat_p=hat_p,
                    Gamma=Gamma,
                    b_node=b_node,
                    b_supply=b_supply,
                    adversary_type=adv,
                    epsilon=epsilon,
                    max_iters=max_iters,
                    gurobi_output=gurobi_output,
                )
                runtime = time.perf_counter() - t0

                rows.append(
                    dict(
                        dataset=os.path.basename(folder),
                        file=fn,
                        n=n,
                        Gamma=Gamma,
                        adversary=adv,
                        runtime=runtime,
                        iters=len(LB_hist),
                        rho_star=float(rho_star) if rho_star is not None else None,
                        best_policy=best_policy,
                    )
                )

    return pd.DataFrame(rows)


# ============================================================
# 6) example main (single folder) + txt output
# ============================================================

if __name__ == "__main__":
    folder = "js30path"  # change to js60path/js90path
    gammas = (3, 5, 7)

    df = benchmark_meta(folder, gammas=gammas, epsilon=1e-3, max_iters=20, gurobi_output=False)
    print(df)

    out_txt = f"meta_results_{os.path.basename(folder)}.txt"
    write_meta_results_txt(df, out_txt)


Restricted license - for non-production use only - expires 2026-11-23


GurobiError: Model too large for size-limited license; visit https://gurobi.com/unrestricted for more information

**EXAMPLES**

In [51]:
import math

# Tiny toy instance
V = [0, 1, 2, 3]                 # 0 = source, 3 = sink
E = [(0, 1), (1, 2), (2, 3)]     # fixed precedence path

# One resource
R = [0]

# Nominal durations bar_p and deviations hat_p
bar_p = [0.0, 3.0, 2.0, 0.0]     # durations for nodes 0..3
hat_p = [0.0, 1.0, 0.5, 0.0]     # deviations for nodes 0..3

# Uncertainty budget Gamma
Gamma = 1.5

# Flow balances b_{ik} (here as list-of-lists [i][k])
# internal nodes (1 and 2) have 1 unit in and out; source/sink have 0
b_node = [
    [0.0],   # i = 0 (source)
    [1.0],   # i = 1 (internal)
    [1.0],   # i = 2 (internal)
    [0.0],   # i = 3 (sink)
]

# Total supply b_k from source to sink for resource k=0
b_supply = [1.0]


In [52]:
best_policy, rho_star, LB_hist, UB_hist, scenarios = worst_case_rcpsp_meta(
    V=V,
    E=E,
    R=R,
    bar_p=bar_p,
    hat_p=hat_p,
    Gamma=Gamma,
    b_node=b_node,
    b_supply=b_supply,
    epsilon=1e-3,
    M_prec=1e4,
    M_flow=1e4,
    max_iters=10,
    gurobi_output=False,   # אפשר True אם אתה רוצה לראות לוגים של גורובי
)

print("=== FINAL RESULT ===")
print("Best policy X* (edges):", best_policy)
print("Worst-case makespan rho*:", rho_star)
print("Scenarios used (P_hat_q):")
for h, p in enumerate(scenarios):
    print(f"  h={h}: p^{h+1} = {p}")

print("\nLB / UB history:")
for t, (lb, ub) in enumerate(zip(LB_hist, UB_hist), start=1):
    print(f"  iter {t}: LB={lb:.4f}, UB={ub:.4f}, gap={ub - lb:.4f}")


Iter 1: selection arcs (policy X^1) = [(0, 1), (1, 2), (2, 3)]
Iter 2: selection arcs (policy X^2) = [(0, 1), (1, 2), (2, 3)]
=== FINAL RESULT ===
Best policy X* (edges): [(0, 1), (1, 2), (2, 3)]
Worst-case makespan rho*: 6.25
Scenarios used (P_hat_q):
  h=0: p^1 = [0.0, 3.0, 2.0, 0.0]
  h=1: p^2 = [0, 4.0, 2.25, 0]

LB / UB history:
  iter 1: LB=5.0000, UB=6.2500, gap=1.2500
  iter 2: LB=6.2500, UB=6.2500, gap=0.0000


In [53]:
# Example data: 6 nodes (0..5), 2 resources, less degenerate edge set

V = [0, 1, 2, 3, 4, 5]   # 0 = source, 5 = sink
source = 0
sink = 5

# Less degenerate precedence arcs: branching & merging
# 0 → 1 → 3 → 5
#  \        ↑
#   → 2 → 4 ┘
#   \→ 3
E = [
    (0, 1),
    (0, 2),
    (1, 3),
    (2, 3),
    (2, 4),
    (3, 5),
    (4, 5),
]

# Nominal durations and deviations
bar_p = [0.0, 3.0, 2.5, 4.0, 1.5, 0.0]
hat_p = [0.0, 1.0, 0.8, 0.5, 0.7, 0.0]

Gamma = 2.2

# Two resources
R = [0, 1]

# Node-by-resource balances b_{ik}
b_node = [
    [0.0, 0.0],   # node 0 (source)
    [1.0, 2.0],   # node 1
    [1.0, 2.0],   # node 2
    [0.0, 1.0],   # node 3
    [1.0, 1.0],   # node 4
    [0.0, 0.0],   # node 5 (sink)
]

# Total supply per resource
b_supply = [1.0, 2.0]

# Run the meta-algorithm (assumes all functions are already defined)
best_policy, rho_star, LB_hist, UB_hist, scenarios = worst_case_rcpsp_meta(
    V=V,
    E=E,
    R=R,
    bar_p=bar_p,
    hat_p=hat_p,
    Gamma=Gamma,
    b_node=b_node,
    b_supply=b_supply,
    epsilon=1e-3,
    M_prec=1e4,
    M_flow=1e4,
    max_iters=10,
    gurobi_output=False,
)

print("=== FINAL RESULT ===")
print("Best policy X* (edges):", best_policy)
print("Worst-case makespan rho*:", rho_star)

print("\nScenarios used (P_hat_q):")
for h, p in enumerate(scenarios):
    print(f"  h={h}: p^{h+1} = {p}")

print("\nLB / UB history:")
for t, (lb, ub) in enumerate(zip(LB_hist, UB_hist), start=1):
    print(f"  iter {t}: LB={lb:.4f}, UB={ub:.4f}, gap={ub - lb:.4f}")


Iter 1: selection arcs (policy X^1) = [(0, 1), (0, 2), (0, 5), (1, 2), (1, 3), (1, 5), (2, 3), (2, 4), (2, 5), (3, 5), (4, 5)]
Iter 2: selection arcs (policy X^2) = [(0, 1), (0, 2), (0, 5), (1, 2), (1, 3), (1, 5), (2, 3), (2, 4), (2, 5), (3, 5), (4, 5)]
=== FINAL RESULT ===
Best policy X* (edges): [(0, 1), (0, 2), (0, 5), (1, 2), (1, 3), (1, 5), (2, 3), (2, 4), (2, 5), (3, 5), (4, 5)]
Worst-case makespan rho*: 11.399999999999999

Scenarios used (P_hat_q):
  h=0: p^1 = [0.0, 3.0, 2.5, 4.0, 1.5, 0.0]
  h=1: p^2 = [0, 4.0, 3.3, 4.1, 0, 0]

LB / UB history:
  iter 1: LB=9.5000, UB=11.4000, gap=1.9000
  iter 2: LB=11.4000, UB=11.4000, gap=-0.0000


In [54]:
# Example data: 6 nodes (0..5), 2 resources, less degenerate edge set

V = [0, 1, 2, 3, 4, 5]   # 0 = source, 5 = sink
source = 0
sink = 5

# Less degenerate precedence arcs: branching & merging
# 0 → 1 → 3 → 5
#  \        ↑
#   → 2 → 4 ┘
#   \→ 3
E = [
    (0, 1),
    (0, 2),
    (1, 3),
    (2, 3),
    (2, 4),
    (3, 5),
    (4, 5),
]

# Nominal durations and deviations
bar_p = [0.0, 3.0, 2.5, 4.0, 1.5, 0.0]
hat_p = [0.0, 1.0, 0.8, 0.5, 3.7, 0.0]

Gamma = 2.2

# Two resources
R = [0, 1]

# Node-by-resource balances b_{ik}
b_node = [
    [0.0, 0.0],   # node 0 (source)
    [1.0, 2.0],   # node 1
    [1.0, 2.0],   # node 2
    [0.0, 1.0],   # node 3
    [1.0, 1.0],   # node 4
    [0.0, 0.0],   # node 5 (sink)
]

# Total supply per resource
b_supply = [1.0, 2.0]

# Run the meta-algorithm (assumes all functions are already defined)
best_policy, rho_star, LB_hist, UB_hist, scenarios = worst_case_rcpsp_meta(
    V=V,
    E=E,
    R=R,
    bar_p=bar_p,
    hat_p=hat_p,
    Gamma=Gamma,
    b_node=b_node,
    b_supply=b_supply,
    epsilon=1e-3,
    M_prec=1e4,
    M_flow=1e4,
    max_iters=10,
    gurobi_output=False,
)

print("=== FINAL RESULT ===")
print("Best policy X* (edges):", best_policy)
print("Worst-case makespan rho*:", rho_star)

print("\nScenarios used (P_hat_q):")
for h, p in enumerate(scenarios):
    print(f"  h={h}: p^{h+1} = {p}")

print("\nLB / UB history:")
for t, (lb, ub) in enumerate(zip(LB_hist, UB_hist), start=1):
    print(f"  iter {t}: LB={lb:.4f}, UB={ub:.4f}, gap={ub - lb:.4f}")


Iter 1: selection arcs (policy X^1) = [(0, 1), (0, 2), (0, 5), (1, 2), (1, 3), (1, 5), (2, 3), (2, 4), (2, 5), (3, 5), (4, 5)]
Iter 2: selection arcs (policy X^2) = [(0, 1), (0, 2), (0, 5), (1, 2), (1, 3), (1, 5), (2, 3), (2, 4), (2, 5), (3, 5), (4, 5)]
=== FINAL RESULT ===
Best policy X* (edges): [(0, 1), (0, 2), (0, 5), (1, 2), (1, 3), (1, 5), (2, 3), (2, 4), (2, 5), (3, 5), (4, 5)]
Worst-case makespan rho*: 11.86

Scenarios used (P_hat_q):
  h=0: p^1 = [0.0, 3.0, 2.5, 4.0, 1.5, 0.0]
  h=1: p^2 = [0, 4.0, 2.66, 0, 5.2, 0]

LB / UB history:
  iter 1: LB=9.5000, UB=11.8600, gap=2.3600
  iter 2: LB=11.8600, UB=11.8600, gap=-0.0000


In [55]:
# Example data: 6 nodes (0..5), 2 resources, less degenerate edge set

V = [0, 1, 2, 3, 4, 5]   # 0 = source, 5 = sink
source = 0
sink = 5

# Less degenerate precedence arcs: branching & merging
# 0 → 1 → 3 → 5
#  \        ↑
#   → 2 → 4 ┘
#   \→ 3
E = [
    (0, 1),
    (0, 2),
    (1, 3),
    (2, 3),
    (2, 4),
    (3, 5),
    (4, 5),
]

# Nominal durations and deviations
bar_p = [0.0, 3.0, 2.5, 4.0, 1.5, 0.0]
hat_p = [0.0, 1.0, 0.8, 0.2, 1.7, 0.0]

Gamma = 2.2

# Two resources
R = [0, 1]

# Node-by-resource balances b_{ik}
b_node = [
    [0.0, 0.0],   # node 0 (source)
    [1.0, 1.0],   # node 1
    [1.0, 2.0],   # node 2
    [0.0, 1.0],   # node 3
    [1.0, 1.0],   # node 4
    [0.0, 0.0],   # node 5 (sink)
]

# Total supply per resource
b_supply = [2.0, 2.0]

# Run the meta-algorithm (assumes all functions are already defined)
best_policy, rho_star, LB_hist, UB_hist, scenarios = worst_case_rcpsp_meta(
    V=V,
    E=E,
    R=R,
    bar_p=bar_p,
    hat_p=hat_p,
    Gamma=Gamma,
    b_node=b_node,
    b_supply=b_supply,
    epsilon=1e-3,
    M_prec=1e4,
    M_flow=1e4,
    max_iters=10,
    gurobi_output=False,
)

print("=== FINAL RESULT ===")
print("Best policy X* (edges):", best_policy)
print("Worst-case makespan rho*:", rho_star)

print("\nScenarios used (P_hat_q):")
for h, p in enumerate(scenarios):
    print(f"  h={h}: p^{h+1} = {p}")

print("\nLB / UB history:")
for t, (lb, ub) in enumerate(zip(LB_hist, UB_hist), start=1):
    print(f"  iter {t}: LB={lb:.4f}, UB={ub:.4f}, gap={ub - lb:.4f}")


Iter 1: selection arcs (policy X^1) = [(0, 1), (0, 2), (0, 5), (1, 2), (1, 3), (1, 5), (2, 3), (2, 4), (3, 5), (4, 5)]
Iter 2: selection arcs (policy X^2) = [(0, 1), (0, 2), (0, 5), (1, 2), (1, 3), (1, 5), (2, 3), (2, 4), (3, 5), (4, 5)]
=== FINAL RESULT ===
Best policy X* (edges): [(0, 1), (0, 2), (0, 5), (1, 2), (1, 3), (1, 5), (2, 3), (2, 4), (3, 5), (4, 5)]
Worst-case makespan rho*: 11.34

Scenarios used (P_hat_q):
  h=0: p^1 = [0.0, 3.0, 2.5, 4.0, 1.5, 0.0]
  h=1: p^2 = [0, 4.0, 3.3, 4.04, 0, 0]

LB / UB history:
  iter 1: LB=9.5000, UB=11.3400, gap=1.8400
  iter 2: LB=11.3400, UB=11.3400, gap=-0.0000
